In [1]:
import sys
from pathlib import Path


# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import *
from spytial.annotations import *
from spytial.annotations import flag


# Binary Search Trees

In [2]:
# Convention: Nodes keys are visualized within the node.
@attribute(field="key")
# Conception:  Left child is to the left and BELOW a node.
@orientation(selector='{ x, y : BSTNode | (y.key not in NoneType) and x.left = y }',
             directions=['below','left'])
# Conception: Right child is to the right and BELOW a node.
@orientation(selector='{ x, y : BSTNode | (y.key not in NoneType) and x.right = y }',
             directions=['below','right'])
# Convention / Viz : Highlight nodes with left child violating BST property in red
@atomColor(selector='{ x : BSTNode | (x.left.key not in NoneType) and (@num:(x.left.key) > @num:(x.key)) }', value='red')
@atomColor(selector='{ x : BSTNode | (x.right.key not in NoneType) and (@num:(x.right.key) < @num:(x.key)) }', value='red')
class BSTNode:
    def __init__(self, key=None, left=None, right=None, parent=None):
        self.key = key
        self.left = left
        self.right = right
        self.parent = parent

# Singleton NIL sentinel
BST_NIL = BSTNode(key=None)
BST_NIL.left = BST_NIL.right = BST_NIL.parent = BST_NIL

@flag(name='hideDisconnected')
# Convention: Hiding extra details for clarity
@hideAtom(selector='{ x : BSTNode | (x.key in NoneType) }') 
@hideAtom(selector='BSTree')   
@hideField(field='parent')  
class BSTree:
    def __init__(self):
        self.root = BST_NIL

    # TREE-SEARCH
    def search(self, k):
        x = self.root
        while x is not BST_NIL and k != x.key:
            x = x.left if k < x.key else x.right
        return x

    # TREE-MINIMUM / MAXIMUM
    def minimum(self, x):
        while x.left is not BST_NIL:
            x = x.left
        return x
    def maximum(self, x):
        while x.right is not BST_NIL:
            x = x.right
        return x

    # TREE-SUCCESSOR
    def successor(self, x):
        if x.right is not BST_NIL:
            return self.minimum(x.right)
        y = x.parent
        while y is not BST_NIL and x is y.right:
            x, y = y, y.parent
        return y

    # TREE-INSERT
    def insert(self, k):
        z = BSTNode(key=k, left=BST_NIL, right=BST_NIL, parent=None)
        y, x = BST_NIL, self.root
        while x is not BST_NIL:
            y = x
            x = x.left if z.key < x.key else x.right
        z.parent = y
        if y is BST_NIL: self.root = z
        elif z.key < y.key: y.left = z
        else: y.right = z
        return z

    # helper: TRANSPLANT
    def _transplant(self, u, v):
        if u.parent is BST_NIL: self.root = v
        elif u is u.parent.left: u.parent.left = v
        else: u.parent.right = v
        v.parent = u.parent

    # TREE-DELETE
    def delete(self, z):
        if z.left is BST_NIL:
            self._transplant(z, z.right)
        elif z.right is BST_NIL:
            self._transplant(z, z.left)
        else:
            y = self.minimum(z.right)     # successor
            if y.parent is not z:
                self._transplant(y, y.right)
                y.right = z.right
                y.right.parent = y
            self._transplant(z, y)
            y.left = z.left
            y.left.parent = y

    # inorder (for sanity checks)
    def inorder(self, node=None, acc=None):
        if node is None:
            node, acc = self.root, []
        if node is BST_NIL:
            return acc
        self.inorder(node.left, acc)
        acc.append(node.key)
        self.inorder(node.right, acc)
        return acc


![bst](/demos/clrs/img/binary-search-tree.png)


In [ ]:

t = BSTree()

# insert keys (any iterable of comparable keys works)
for k in [15, 6, 18, 17, 20, 3, 7, 13, 9, 2, 4]:
    t.insert(k)

diagram(t) # Shows you each step of the insertion



# def build_unordered_tree():
#     # Build some bst nodes
#     t = BSTree()
#     n1 = t.insert(10)
#     n2 = t.insert(5)
#     n3 = t.insert(15)
#     n4 = t.insert(20)
#     # Then connect them incorrectly (as a tree but out of order)
#     n1.left = n3; n3.parent = n1
#     n1.right = n2; n2.parent = n1
#     n2.right = n4; n4.parent = n2
#     n3.right = BST_NIL

#     return t

# inct = build_unordered_tree()
# diagram(inct)  # Visualize the incorrect tree structure

# Red Black Trees

### CLRS Image

![rbt-3-ways](/Users/siddharthaprasad/Desktop/SpatialRefinement/cnd-py/demos/clrs/img/red-black-tree.png)

In [4]:

t = RBTree()


## TODO: make sure the order of insertion is correct to recreate the img?
for k in [26, 17, 41, 47, 30, 38, 35, 39, 28, 21, 23, 19, 20, 14, 16, 15, 10, 7, 3, 12]:
    #print(f"Inserting {k}...")
    t.insert(k)
diagram(t) # Shows you each step of the insertion




NameError: name 'RBTree' is not defined

In [ ]:
from typing import List

@hideAtom(selector='MaxHeap')
# infer left child
@inferredEdge(selector="{ a, b : int | (some xs : list | (some i : int | xs->i->a in idx  and xs->multiply[@num:i,2]->b in idx  )   )}", name = "left")
class MaxHeap:
    """
    CLRS-style max heap storing integers.
    1-indexed: a[0] unused.
    """
    def __init__(self, data: List[int] = None):
        self.a: List[int] = [0]
        if data:
            self.a.extend(data)
        self.n = len(self.a) - 1
        if self.n > 1:
            self.build_max_heap()

    # index helpers
    @staticmethod
    def _parent(i: int) -> int: return i // 2
    @staticmethod
    def _left(i: int) -> int:   return 2 * i
    @staticmethod
    def _right(i: int) -> int:  return 2 * i + 1

    def _max_heapify(self, i: int) -> None:
        while True:
            l, r = self._left(i), self._right(i)
            largest = i
            if l <= self.n and self.a[l] > self.a[largest]:
                largest = l
            if r <= self.n and self.a[r] > self.a[largest]:
                largest = r
            if largest == i:
                break
            self.a[i], self.a[largest] = self.a[largest], self.a[i]
            i = largest

    def build_max_heap(self) -> None:
        for i in range(self.n // 2, 0, -1):
            self._max_heapify(i)

    # API
    def max(self) -> int:
        if self.n < 1:
            raise IndexError("heap underflow")
        return self.a[1]

    def extract_max(self) -> int:
        if self.n < 1:
            raise IndexError("heap underflow")
        m = self.a[1]
        self.a[1] = self.a[self.n]
        self.a.pop()
        self.n -= 1
        if self.n >= 1:
            self._max_heapify(1)
        return m

    def increase_key(self, i: int, key: int) -> None:
        if i < 1 or i > self.n:
            raise IndexError("index out of range")
        if key < self.a[i]:
            raise ValueError("new key is smaller than current key")
        self.a[i] = key
        while i > 1 and self.a[self._parent(i)] < self.a[i]:
            p = self._parent(i)
            self.a[i], self.a[p] = self.a[p], self.a[i]
            i = p

    def insert(self, key: int) -> None:
        self.n += 1
        self.a.append(float("-inf"))  # sentinel
        self.increase_key(self.n, key)

    def __len__(self) -> int:
        return self.n

    def __repr__(self) -> str:
        return f"MaxHeap({self.a[1:]})"





In [ ]:
# Example
h = MaxHeap([16, 14, 10, 8, 7, 9, 3, 2, 4, 1])
diagram(h)

evaluate(h)



# Fibonacci Heaps (priority queues)

- Roots of all trees are maintained in a ircular doubly linked list

In [ ]:
import math

## BUGGY -- this is wrong I think (?)

# Fibonacci Heap (priority queue) with spatial annotations for visualization
@attribute(field="key")
@attribute(field="degree")
@attribute(field="mark")

@cyclic(selector='{ disj x, y : FibNode | x->y in right}', direction='clockwise')

# @orientation(selector='{ x, y : FibNode | (y.key not in NoneType) and y.parent = x }',              directions=['below'])
# @orientation(selector='{ x, y : FibNode | (x.parent in FibNode) and (y.key not in NoneType) and x.right = y }',             directions=['right'])
# @orientation(selector='{ x, y : FibNode | (x.parent in FibNode) and (y.key not in NoneType) and x.left = y }',              directions=['left'])
# @atomColor(selector='{ x : FibNode | @bool:(x.mark) }', value='red')
class FibNode:
    def __init__(self, key=None):
        self.key = key
        self.degree = 0
        self.mark = False
        self.parent = None
        self.child = None
        # circular doubly-linked list pointers
        self.left = self
        self.right = self

@flag(name='hideDisconnected')
@attribute(field="n")
@hideAtom(selector='NoneType')
# @hideAtom(selector='FibonacciHeap')   # hide wrapper
# @hideField(field='parent')            # hide parent pointers in visualization
class FibonacciHeap:
    def __init__(self):
        self.min: FibNode | None = None
        self.n: int = 0

    # helper: iterate a circular doubly-linked list starting at node
    def _iterate(self, start: FibNode):
        if start is None:
            return
        node = start
        while True:
            yield node
            node = node.right
            if node is start:
                break

    # insert a new key, return the node
    def insert(self, key):
        x = FibNode(key=key)
        # add to root list
        if self.min is None:
            self.min = x
        else:
            # insert x to the right of min
            x.right = self.min.right
            x.left = self.min
            self.min.right.left = x
            self.min.right = x
            if x.key < self.min.key:
                self.min = x
        self.n += 1
        return x

    def find_min(self):
        return self.min

    # merge another heap into this one (destructive)
    def union(self, other: "FibonacciHeap"):
        if other is None or other.min is None:
            return
        if self.min is None:
            self.min = other.min
            self.n = other.n
            return
        # concatenate root lists
        a = self.min.right
        b = other.min.left
        self.min.right = other.min
        other.min.left = self.min
        a.left = b
        b.right = a
        if other.min.key < self.min.key:
            self.min = other.min
        self.n += other.n

    # extract the minimum node and return it
    def extract_min(self):
        z = self.min
        if z is not None:
            # move z's children to root list
            if z.child is not None:
                children = list(self._iterate(z.child))
                for x in children:
                    # remove parent link
                    x.parent = None
                    # splice x into root list (to the right of min)
                    x.left = self.min
                    x.right = self.min.right
                    self.min.right.left = x
                    self.min.right = x
            # remove z from root list
            if z is z.right:
                self.min = None
            else:
                z.left.right = z.right
                z.right.left = z.left
                self.min = z.right
                self._consolidate()
            self.n -= 1
        return z

    def _consolidate(self):
        if self.min is None:
            return
        # upper bound on degree
        max_degree = int(math.log(self.n, 2)) + 2 if self.n > 0 else 1
        A = [None] * (max_degree + 1)
        roots = list(self._iterate(self.min))
        for w in roots:
            x = w
            d = x.degree
            while A[d] is not None:
                y = A[d]
                if x.key > y.key:
                    x, y = y, x
                self._link(y, x)
                A[d] = None
                d += 1
            A[d] = x
        # rebuild root list and find new min
        self.min = None
        for node in A:
            if node is not None:
                # isolate node
                node.left = node.right = node
                if self.min is None:
                    self.min = node
                else:
                    # insert to root list
                    node.right = self.min.right
                    node.left = self.min
                    self.min.right.left = node
                    self.min.right = node
                    if node.key < self.min.key:
                        self.min = node

    def _link(self, y: FibNode, x: FibNode):
        # remove y from root list
        y.left.right = y.right
        y.right.left = y.left
        # make y a child of x
        y.parent = x
        y.left = y.right = y
        if x.child is None:
            x.child = y
        else:
            # insert y into x's child circular list
            y.right = x.child.right
            y.left = x.child
            x.child.right.left = y
            x.child.right = y
        x.degree += 1
        y.mark = False

    def decrease_key(self, x: FibNode, k):
        if k > x.key:
            raise ValueError("new key is greater than current key")
        x.key = k
        y = x.parent
        if y is not None and x.key < y.key:
            self._cut(x, y)
            self._cascading_cut(y)
        if x.key < self.min.key:
            self.min = x

    def _cut(self, x: FibNode, y: FibNode):
        # remove x from y's child list
        if y.child is x:
            # if x is the only child
            if x.right is x:
                y.child = None
            else:
                y.child = x.right
        x.left.right = x.right
        x.right.left = x.left
        y.degree -= 1
        # add x to root list
        x.left = self.min
        x.right = self.min.right
        self.min.right.left = x
        self.min.right = x
        x.parent = None
        x.mark = False

    def _cascading_cut(self, y: FibNode):
        z = y.parent
        if z is not None:
            if not y.mark:
                y.mark = True
            else:
                self._cut(y, z)
                self._cascading_cut(z)

    def delete(self, x: FibNode):
        # decrease key to -infinity then extract min
        self.decrease_key(x, -float("inf"))
        self.extract_min()

# Demo: insert keys and visualize after each step
t = FibonacciHeap()
for k in [10, 20, 30, 15, 25, 5, -10, 0]:
    t.insert(k)

diagram(t)

# van Emde Boas (vEB) Tree (CLRS Ch. 20)

TODO

# MST

TODO

- Kruskals
- Prims's